<div style="background-image:url(images/meschede-seismic-waves.png); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px; background-position:0px -250px">
<p style="float:right; margin-top:20px; padding: 10px 20px 0px 20px; background:rgba(255,255,255,0.75); border-radius:10px;">
<img width="400px" src=images/obspy_logo.png>
</p>

<h2 style="color:#bbb">IRIS WEBINAR</h2>

<h1 style="color:#EEE">ObsPy: A Python Toolbox for Seismology</h1>

<h4 style="color:#FFF">May 26th, 2015</h4>

http://www.obspy.org
</div>

If you want to follow along, download and unpack

https://raw.githubusercontent.com/obspy/docs/master/notebooks/IRIS_Webinar/webinar.tar.gz

or execute the following lines:


```bash
$ mkdir iris_webinar
$ cd iris_webinar
$ curl -k https://raw.githubusercontent.com/obspy/docs/master/notebooks/IRIS_Webinar/webinar.tar.gz | \
  tar xzvf -
$ ipython notebook IRIS_Webinar.ipynb
```

If you don't yet have ObsPy and the IPython notebook installed, please do it by following the instructions on our website (http://www.obspy.org). For a quick installation, download Anaconda (http://continuum.io/downloads), and execute

```bash
$ conda install -c obspy obspy ipython-notebook
```

---

The next couple of lines do three things:
* Make plots appear in the browser (otherwise a window pops up)
* Printing things works like this: 

```python
print("Hello")
```

* Plots look a bit nicer and bigger by default.

This essentially makes this notebook work under Python 2 and Python 3.

In [ ]:
from __future__ import print_function
import matplotlib.pylab as plt
plt.switch_backend("nbagg")
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

---

## Waveform Data

![stream](images/Stream_Trace.svg)

In [ ]:
ls -l waveforms

In [ ]:
import obspy

st = obspy.read("waveforms/data.mseed")

print(st)

In [ ]:
st2 = obspy.read("./waveforms/data_*.sac")

print(st2)

In [ ]:
tr = st[0]

print(tr.stats)

tr.plot();

In [ ]:
tr.detrend("linear")
tr.write("test.sac", format="sac")

## Station Data

Station information used to (and still is) served as SEED or RESP files; ObsPy can deal with both. The modern format is StationXML.

In [ ]:
import obspy

inv = obspy.read_inventory("./stations/TA_527A.xml")
print(inv)

### Structure

![inv](images/Inventory.svg)

In [ ]:
print(inv.select(channel="*Z"))

In [ ]:
inv.select(channel="*Z").write("z_component.xml", format="stationxml")

In [ ]:
!head -n 10 z_component.xml

In [ ]:
inv.plot_response(min_freq=0.001);

## Event Data

ObsPy can read all kinds of event data formats, like QuakeML (which is shown later down this notebook) and the NDK format of the GCMT Catalog: http://www.globalcmt.org/CMTfiles.html

In [ ]:
import obspy

cat = obspy.readEvents("./events/2014.ndk")

print(cat)

In [ ]:
cat.plot();

In [ ]:
cat.filter("depth > 100000", "magnitude > 7")

In [ ]:
cat.filter("magnitude > 7.5")

In [ ]:
cat.filter("magnitude > 7.5").write("test.xml", format="quakeml")

In [ ]:
!head -n 10 test.xml

### Structure

Base on QuakeML.

![events](./images/Event.svg)

In [ ]:
event = cat[0]
print(event)

In [ ]:
print(event.preferred_origin() or event.origins[0])

In [ ]:
magnitude = event.preferred_magnitude() or event.magnitude[0]
print(magnitude.mag)

## Webservices

In [ ]:
import obspy
from obspy.fdsn import Client

c = Client("IRIS")
print(c)

In [ ]:
c.get_stations(network="TA", level="station").plot(projection="local");

## Exercise

For the next exercise we will download some data from the Tohoku-Oki earthquake, cut out a certain time window around the first arrival and remove the instrument response from the data.

In [ ]:
import obspy
from obspy.fdsn import Client

c = Client("IRIS")

# Event time.
event_time = obspy.UTCDateTime("2011-03-11T05:46:23.2")


cat = c.get_events(starttime=event_time - 10, endtime=event_time + 10,
                   minmagnitude=9)

inv = c.get_stations(network="TA", station="637A", level="response")

st = c.get_waveforms(network="TA", station="637A", location="",
                     channel="BH?", starttime=event_time - 60,
                     endtime=event_time + 3600)

In [ ]:
st.plot();

In [ ]:
coords = inv.get_coordinates("TA.637A..BHE")
coords

In [ ]:
from obspy.core.util.geodetics import locations2degrees

origin = cat[0].preferred_origin()
distance = locations2degrees(origin.latitude, origin.longitude,
                             coords["latitude"], coords["longitude"])

In [ ]:
from obspy.taup import TauPyModel

m = TauPyModel(model="ak135")

arrivals = m.get_ray_paths(
    distance_in_degree=distance,
    source_depth_in_km=origin.depth / 1000.0)

arrivals.plot();

In [ ]:
# Cut 1 minute before and 5 minute after first arrivals.
first_arrival = origin.time + arrivals[0].time
st2 = st.slice(first_arrival - 60, first_arrival + 300)
st2.plot();

<img width=75% src=images/taper.png></img>

In [ ]:
st3 = st2.copy()
st3.attach_response(inv)
st3.remove_response(pre_filt=(1.0 / 10.0, 1.0 / 5.0, 1.0, 2.0),
                    output="VEL")
st3.plot()

## Bonus: Interactive IPython widgets

In [ ]:
from IPython.html.widgets import interact
from obspy.taup import TauPyModel

m = TauPyModel("ak135")

def plot_raypaths(distance, depth, wavetype):
    try:
        plt.close()
    except:
        pass
    if wavetype == "all":
        phases = ["ttall"]
    elif wavetype == "diff":
        phases = ["Pdiff", "pPdiff"]
    m.get_ray_paths(distance_in_degree=distance,
                    source_depth_in_km=depth,
                    phase_list=phases).plot();
    
interact(plot_raypaths, distance=[0, 180],
         depth=[0, 700], wavetype=["all", "diff"])

## Acknowledgements

Background picture at the very top is from Matthias Meschede.